## Classificação de imagens com redes neurais convolucionais

Bem-vindos à oficina de Deep Learning - Visão Computacional.     
Vamos usar redes neurais por convolução (CNNs) para ensinar o computador reconhecer imagens.   
Algo possível graças ao aprendizado profundo(deep learning).

## Introdução ao Deep Learning: 'Cães e Gatos'

Vamos criar um modelo para entrar na competição Dogs vs Cats no Kaggle.   
Temos 25.000 fotos de cães e gatos rotuladas disponíveis para treinamento e   
12.500 no conjunto de testes que devemos tentar rotular para esta competição.   
De acordo com o site da Kaggle, quando esta competição foi lançada (final de 2013): "Estado da arte: A literatura atual sugere que classificadores de máquinas podem pontuar acima de 80% de precisão nesta tarefa".  
  
Então, se conseguirmos bater 80%, estaremos na vanguarda a partir de 2013!


In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Vamos importar as bibliotecas, com código aberto, que vamos precisar.

In [ ]:
# This file contains all the main external libs we'll use
from fastai.imports import *

In [ ]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

`PATH` é o caminho para seus dados - se você usar as abordagens de configuração recomendadas da lição, não precisará alterar isso. `sz` é o tamanho que as imagens serão redimensionadas para garantir que o treinamento seja executado rapidamente. Nós estaremos falando muito sobre este parâmetro durante o curso. Deixe-o em `224` por enquanto.


In [ ]:
PATH = "../input/"
TMP_PATH = "/tmp/tmp"
MODEL_PATH = "/tmp/model/"
sz=224

É importante que você tenha uma GPU NVidia em funcionamento. A estrutura de programação usada nos bastidores para trabalhar com GPUs NVidia é chamada de CUDA. Portanto, você precisa garantir que a linha a seguir retorne `True` antes de prosseguir. Se você tiver problemas com isso, verifique o FAQ e peça ajuda nos [fóruns] (http://forums.fast.ai).


In [ ]:
torch.cuda.is_available()

Além disso, a NVidia oferece funções  especialmente aceleradas para aprendizado profundo em um pacote chamado CuDNN. Embora não seja estritamente necessário, ele melhorará significativamente o desempenho do treinamento e será incluído por padrão em todas as configurações fastai suportadas. Portanto, se o seguinte não retornar True, você pode querer investigar o motivo.


In [ ]:
torch.backends.cudnn.enabled

## Vamos conhecer as fotos dos gatos

A biblioteca Fastai assumirá que você tem diretórios de treino e teste. Ele também assume que cada diretório terá subdiretórios para cada classe que você deseja reconhecer (neste caso, 'cats' e 'dogs').


In [ ]:
PATH

In [ ]:
os.listdir(PATH)

In [ ]:
fnames = np.array([f'train/{f}' for f in sorted(os.listdir(f'{PATH}train'))])
labels = np.array([(0 if 'cat' in fname else 1) for fname in fnames])

In [ ]:
len(os.listdir(f'{PATH}train'))

In [ ]:
len(os.listdir(f'{PATH}test'))

In [ ]:
os.listdir(f'{PATH}test')[:5]

In [ ]:
img = plt.imread(f'{PATH}{fnames[1]}')
plt.imshow(img);
#files = os.listdir(f'{PATH}test/cats')[:5]
#files

In [ ]:
# img = plt.imread(f'{PATH}valid/cats/{files[1]}')
# plt.imshow(img);

Here is how the raw data looks like

In [ ]:
img.shape

In [ ]:
img[:4,:4]

## Nosso primeiro modelo - início rápido.


Vamos usar um modelo pré-treinado, ou seja, um modelo criado por alguém para resolver um problema diferente. Em vez de construir um modelo a partir do zero para resolver um problema semelhante, usaremos um modelo treinado no ImageNet (1,2 milhões de imagens e 1000 classes) como ponto de partida. O modelo é uma rede neural por convolução (CNN), um tipo de rede neural que constrói modelos de última geração para visão computacional.   

Vamos usar o modelo resnet34. O resnet34 é uma versão do modelo que ganhou a competição 2015 ImageNet.  

Veja como treinar e avaliar um modelo de cães vs gatos em 3 linhas de código e em menos de 2 minutos (no laptop):

In [ ]:
# Uncomment the below if you need to reset your precomputed activations
# shutil.rmtree(f'{PATH}tmp', ignore_errors=True)

In [ ]:
arch=resnet34
# data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))
# learn = ConvLearner.pretrained(arch, data, precompute=True)
data = ImageClassifierData.from_names_and_array(
    path=PATH, 
    fnames=fnames, 
    y=labels, 
    classes=['dogs', 'cats'], 
    test_name='test', 
    tfms=tfms_from_model(arch, sz)
)
learn = ConvLearner.pretrained(arch, data, precompute=True, tmp_name=TMP_PATH, models_name=MODEL_PATH)
learn.fit(0.01, 2)

Será que nosso modelo é bom ?  Bem, como mencionamos, antes desta competição, o estado da arte tinha 80% de precisão. Mas a competição resultou em um enorme salto para 99% de precisão, com o autor de uma popular biblioteca de aprendizagem profunda vencendo a competição. Extraordinariamente, menos de 4 anos depois, podemos agora bater esse resultado em segundos! 


## Entendendo o código do nosso primeiro modelo

Vamos ver o código Dogs v Cats linha por linha.

**tfms** significa *transformations*. `tfms_from_model` cuida do redimensionamento, recorte de imagem, normalização inicial (criação de dados com (mean,stdev) of (0,1)), e outras coisas mais.


In [ ]:
# tfms = tfms_from_model(resnet34, sz)

Precisamos de um <b>path</b> que aponte para o conjunto de dados. Nesse caminho, também armazenaremos dados temporários e resultados finais. `ImageClassifierData.from_paths` lê dados de um caminho fornecido e cria um conjunto de dados pronto para treinamento.


In [ ]:
# data = ImageClassifierData.from_names_and_array(
#     path=PATH, 
#     fnames=fnames, 
#     y=labels, 
#     classes=['dogs', 'cats'], 
#     test_name='test', 
#     tfms=tfms_from_model(arch, sz)
# )

`ConvLearner.pretrained` constrói um *learner* que contém um modelo pré-treinado. A última camada do modelo precisa ser substituída com a camada de dimensões corretas. O modelo pre-treinado foi treinado para 1000 classes, portanto, a camada final prevê um vetor de 1000 probabilidades. O modelo para gatos e cães precisa produzir um vetor bidimensional. O diagrama abaixo mostra em um exemplo como isso foi feito em uma das primeiras CNNs bem-sucedidas. A camada "FC8" aqui seria substituída por uma nova camada com 2 saídas.

<img src="images/pretrained.png" width="500">
[original image](https://image.slidesharecdn.com/practicaldeeplearning-160329181459/95/practical-deep-learning-16-638.jpg)

In [ ]:
# learn = ConvLearner.pretrained(resnet34, data, precompute=True)

Os parâmetros são aprendidos ajustando um modelo aos dados. Os hiperparâmetros são outro tipo de parâmetro, que não podem ser aprendidos diretamente do processo de treinamento regular. Esses parâmetros expressam propriedades de “alto nível” do modelo, como sua complexidade ou quão rápido ele deve aprender. Dois exemplos de hiperparâmetros são a taxa de aprendizado(*learning rate*) e o número de épocas(*number of epochs*).

Durante o treinamento iterativo de uma rede neural, um lote(*batch*) ou mini-lote(*mini-batch*) é um subconjunto de amostras de treinamento usado em uma iteração do Stochastic Gradient Descent (SGD). Uma época é uma passagem única por todo o conjunto de treinamento, que consiste em várias iterações de SGD.

Agora podemos treinar(*fit*) o modelo; isto é, use a descida de gradiente(*gradient descent*) para encontrar os melhores parâmetros para a camada totalmente conectada(fully connected layer) que adicionamos, que pode separar as imagens de gatos das fotos de cães. Precisamos passar dois hyperâmetros: a taxa de aprendizado - *learning rate* (geralmente 1e-2 ou 1e-3 é um bom ponto de partida, veremos mais a seguir) e o número de épocas - *number of epochs* (você pode passar em um número maior e simplesmente parar de treinar quando você vê que não está mais melhorando, então execute-o novamente com o número de épocas que você achou que funciona bem.)



In [ ]:
# learn.fit(1e-2, 1)

## Analisando resultados: olhando as fotos

Além de observar as métricas em geral, também é uma boa ideia ver exemplos de cada uma delas:

1. Algumas classificações corretas aleatoriamente
2. Algumas classificações incorretas aleatoriamente
3. Os rótulos mais corretos de cada classe (ou seja, aqueles com maior probabilidade de estarem corretos)
4. Os rótulos mais incorretos de cada classe (ou seja, aqueles com maior probabilidade de estarem incorretos)
5. Os rótulos mais incertos (isto é, aqueles com probabilidade mais próxima de 0,5).


In [ ]:
# This is the label for a val data
data.val_y

In [ ]:
# from here we know that 'cats' is label 0 and 'dogs' is label 1.
data.classes

In [ ]:
# this gives prediction for validation set. Predictions are in log scale
log_preds = learn.predict()
log_preds.shape

In [ ]:
log_preds[:10]

In [ ]:
preds = np.argmax(log_preds, axis=1)  # from log probabilities to 0 or 1
probs = np.exp(log_preds[:,1])        # pr(dog)

In [ ]:
def rand_by_mask(mask): return np.random.choice(np.where(mask)[0], 4, replace=False)
def rand_by_correct(is_correct): return rand_by_mask((preds == data.val_y)==is_correct)

In [ ]:
def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

In [ ]:
def load_img_id(ds, idx): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8))

In [ ]:
# 1. A few correct labels at random
plot_val_with_title(rand_by_correct(True), "Classificados corretamente")

In [ ]:
# 2. A few incorrect labels at random
plot_val_with_title(rand_by_correct(False), "Classificados incorretamente")

In [ ]:
def most_by_mask(mask, mult):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]

def most_by_correct(y, is_correct): 
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask(((preds == data.val_y)==is_correct) & (data.val_y == y), mult)

In [ ]:
plot_val_with_title(most_by_correct(0, True), "Gatos melhor classificados")

In [ ]:
plot_val_with_title(most_by_correct(1, True), "Cães melhor classificados")

In [ ]:
plot_val_with_title(most_by_correct(0, False), "Gatos pior classificados")

In [ ]:
plot_val_with_title(most_by_correct(1, False), "Cães pior classificados")

In [ ]:
most_uncertain = np.argsort(np.abs(probs -0.5))[:4]
plot_val_with_title(most_uncertain, "Predições mais incertas")

## Escolhendo a taxa de aprendizagem

A taxa de aprendizado determina quão rápido ou quão lento você deseja atualizar os pesos (ou parâmetros). A taxa de aprendizado é um dos parâmetros mais difíceis de definir, porque afeta significativamente o desempenho do modelo.

O método learn.lr_find () ajuda você a encontrar uma taxa de aprendizado ideal. Ele usa a técnica desenvolvida no paper 2015 Cyclical Learning Rates for Training de Redes Neurais, onde nós simplesmente continuamos aumentando a taxa de aprendizado de um valor muito pequeno, até que a perda pare de diminuir. Podemos traçar a taxa de aprendizado entre os lotes(batches) para ver como isso se parece.

Primeiro criamos um novo learner, pois queremos saber como definir a taxa de aprendizado para um novo modelo (não treinado).


In [ ]:
learn = ConvLearner.pretrained(arch, data, precompute=True, tmp_name=TMP_PATH, models_name=MODEL_PATH)

In [ ]:
lrf=learn.lr_find()

Nosso objeto `learn` contém um atributo `sched` que contém nosso scheduler de taxa de aprendizado e possui alguma funcionalidade gráfica interessante, incluindo esta:


In [ ]:
learn.sched.plot_lr()

Observe que na iteração de plotagem anterior há uma iteração (ou minibatch) de SGD(Stochastic gradient descent). Em uma época existem (num_train_samples / num_iterations) de SGD.

Podemos ver o gráfico da perda versus taxa de aprendizado para ver onde nossa função de perda para de diminuir:


In [ ]:
learn.sched.plot()

A perda ainda está claramente melhorando em lr = 1e-2 (0,01), e é isso que usamos. Observe que a taxa de aprendizado ideal pode mudar conforme treinamos o modelo, portanto, você pode querer executar novamente essa função de tempos em tempos.


## Melhorando o modelo

### Dados aumentados (data augmentation)

Se você tentar treinar por mais épocas, perceberá que começamos a ter *overfit*, o que significa que nosso modelo está aprendendo a reconhecer as imagens específicas no conjunto de treinamento, em vez de generalizar, de tal forma que possamos obter bons resultados no conjunto de validação. Uma maneira de corrigir isso é criar, literalmente, mais dados, por meio do *data augmentation*. Isso significa alterar aleatoriamente as imagens de maneira que não venha a afetar sua interpretação, tal como inverter horizontalmente, aplicar zoom e girar.

Podemos fazer isso, passando `aug_tfms` (transformações de aumento-*augmentation transforms*) para `tfms_from_model`, com uma lista de funções a serem aplicadas que alteram aleatoriamente a imagem como desejarmos. Para fotos tiradas em grande parte do lado (por exemplo, a maioria das fotos de cães e gatos, em oposição a fotos tiradas de cima para baixo, como imagens de satélite), podemos usar a lista predefinida de funções `transforms_side_on`. Também podemos especificar o zoom aleatório de imagens até a escala especificada, adicionando o parâmetro `max_zoom`.


In [ ]:
tfms = tfms_from_model(resnet34, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [ ]:
def get_augs():
    data = ImageClassifierData.from_names_and_array(
        path=PATH, 
        fnames=fnames, 
        y=labels, 
        classes=['dogs', 'cats'], 
        test_name='test', 
        tfms=tfms,
        num_workers=1,
        bs=2
    )
    x,_ = next(iter(data.aug_dl))
    return data.trn_ds.denorm(x)[1]

In [ ]:
ims = np.stack([get_augs() for i in range(6)])

In [ ]:
plots(ims, rows=2)

Vamos criar novos objetos de dados que incluam esta augmentation dentro do transforms.


In [ ]:
data = ImageClassifierData.from_names_and_array(
    path=PATH, 
    fnames=fnames, 
    y=labels, 
    classes=['dogs', 'cats'], 
    test_name='test', 
    tfms=tfms
)
learn = ConvLearner.pretrained(arch, data, precompute=True, tmp_name=TMP_PATH, models_name=MODEL_PATH)

In [ ]:
learn.fit(1e-2, 1)

In [ ]:
learn.precompute=False

Por padrão, quando criamos um `learner`, ele define todos, exceto a última camada, para *frozen*. Isso significa que está apenas atualizando os pesos na última camada, quando chamamos o `fit`.


In [ ]:
learn.fit(1e-2, 3, cycle_len=1)

O que é o parâmetro cycle_len? O que fizemos aqui foi usar uma técnica chamada descida de gradiente estocástica com reinicializações - *stochastic gradient descent with restarts (SGDR)*, uma variante de *learning rate annealing*, que diminui gradualmente a taxa de aprendizado à medida que o treinamento avança. Isso é útil porque, à medida que nos aproximamos dos pesos ideais, queremos dar passos menores.

No entanto, podemos nos encontrar em uma parte do espaço de peso que não é muito resiliente - isto é, pequenas mudanças nos pesos podem resultar em grandes mudanças na função de perda. Queremos incentivar nosso modelo a encontrar partes do espaço de peso que sejam precisas e estáveis. Portanto, de tempos em tempos, aumentamos a taxa de aprendizado (isso é o 'restarts' em 'SGDR'), o que forçará o modelo a saltar para uma parte diferente do espaço de peso se a área atual for "spikey". Aqui está uma imagem de como isso pode parecer se redefinirmos as taxas de aprendizado 3 vezes (neste documento eles chamam de "cyclic LR schedule"):  
<img src="images/sgdr.png" width="80%">
(From the paper [Snapshot Ensembles](https://arxiv.org/abs/1704.00109)).

O número de épocas entre a redefinição da taxa de aprendizado é definido por `cycle_len`, e o número de vezes que isso acontece é chamado de *number of cycles*, e é o que estamos passando como o segundo parâmetro para` fit( )`. Então, eis como eram, realmente, as taxas de aprendizado:


In [ ]:
learn.sched.plot_lr()

Nossa perda(loss) na validação não está melhorando muito, então provavelmente não há necessidade de treinar mais a última camada.


Já que temos um modelo muito bom neste momento, podemos querer salvá-lo para que possamos carregá-lo novamente mais tarde sem termos de treiná-lo do zero novamente.


In [ ]:
learn.save('224_lastlayer')

In [ ]:
learn.load('224_lastlayer')

### Ajuste fino e 'differential learning rate annealing'

Agora que temos uma boa camada final treinada, podemos fazer um ajuste fino das outras camadas. Para dizer ao 'learner' que queremos descongelar(unfreeze) as camadas restantes, basta chamar  `unfreeze()`.


In [ ]:
learn.unfreeze()

Observe que as outras camadas já foram treinadas para reconhecer fotos da imagenet (enquanto que nossas camadas finais foram inicializadas aleatoriamente), portanto, queremos ter cuidado para não destruir os pesos, cuidadosamente ajustados, que já estão lá.

De um modo geral, as camadas anteriores (como vimos) têm mais recursos de propósito geral. Portanto, esperamos que elas precisem de menos ajuste fino para novos conjuntos de dados. Por essa razão, usaremos diferentes taxas de aprendizado para diferentes camadas: as primeiras camadas terão 1e-4, as camadas intermediárias em 1e-3 e nossas camadas FC ficarão em 1e-2 como antes. Referimo-nos a isso como *differential learning rates*, embora não haja um nome padrão para essa técnica na literatura, até onde  estamos cientes.


In [ ]:
lr=np.array([1e-4,1e-3,1e-2])

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

Outro truque que usamos aqui, é adicionar o parâmetro cycle_mult. Dê uma olhada no gráfico a seguir e veja se você pode descobrir o que o parâmetro está fazendo:


In [ ]:
learn.sched.plot_lr()

Note que o que está sendo plotado acima é a taxa de aprendizado das camadas finais. As taxas de aprendizado das camadas anteriores são fixadas nos mesmos múltiplos das taxas de camada final como solicitamos inicialmente (ou seja, as primeiras camadas têm 100x menores e as camadas intermediárias 10x menores taxas de aprendizado, pois definimos `lr=np.array([1e-4,1e-3,1e-2]).


In [ ]:
learn.save('224_all')

In [ ]:
learn.load('224_all')

Há algo a mais que podemos fazer com o aumento de dados(data augmentation): use-o no *inference* time (também conhecido como *test* time). Não é de surpreender que isso seja conhecido como * aumento do tempo de teste * ou apenas * TTA *.

O TTA simplesmente faz previsões não apenas sobre as imagens em seu conjunto de validação, mas também faz previsões em um número de versões aumentadas aleatoriamente(randomly augmented versions) delas também (por padrão, ele usa a imagem original junto com 4 versões aumentadas aleatoriamente). Em seguida, faz a previsão média dessas imagens e usa isso. Para usar o TTA no conjunto de validação, podemos usar o método `TTA()` method.


In [ ]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)

In [ ]:
accuracy_np(probs, y)

Geralmente verifica-se uma redução de 10-20% no erro neste conjunto de dados ao usar o TTA neste momento, o que é um resultado incrível para uma técnica tão rápida e fácil!


## Analisando os resultados

### Matrix de confusão

In [ ]:
preds = np.argmax(probs, axis=1)
probs = probs[:,1]

Uma maneira comum de analisar o resultado de um modelo de classificação é usar uma [matriz de confusão](http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/).   
O Scikit-learn possui uma função interessante que podemos usar para esse propósito:


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

Podemos apenas imprimir a matriz de confusão, ou podemos mostrar um gráfico (que é principalmente útil para dependentes com um número maior de categorias).


In [ ]:
plot_confusion_matrix(cm, data.classes)

### Vendo as fotos novamente

In [ ]:
plot_val_with_title(most_by_correct(0, False), "Gatos mais incorretos")

In [ ]:
plot_val_with_title(most_by_correct(1, False), "Cães mais incorretos")

## Revisão : passos fáceis para treinar um classificador de imagem de nível mundial


1. precompute=True
1. Use `lr_find()` to find highest learning rate where loss is still clearly improving
1. Train last layer from precomputed activations for 1-2 epochs
1. Train last layer with data augmentation (i.e. precompute=False) for 2-3 epochs with cycle_len=1
1. Unfreeze all layers
1. Set earlier layers to 3x-10x lower learning rate than next higher layer
1. Use `lr_find()` again
1. Train full network with cycle_mult=2 until over-fitting

## Analisando os resultados: perda(loss) e acurácia(accuracy)

Quando executamos `learn.fit`, imprimimos 3 valores de desempenho (ver acima). Aqui, 0,03 é o valor da perda no conjunto de treinamento, 0,0226 é o valor da perda(**loss**) no conjunto de validação e 0,9927 é a precisão na validação. Qual é a perda? O que é precisão? Por que não apenas mostrar precisão?

Precisão(**Accuracy**) é a razão entre a previsão correta e o número total de previsões.

No aprendizado de máquina, a função de perda(**loss**) ou função de custo, representa o preço pago pela imprecisão das previsões.

A perda associada a um exemplo na classificação binária é dada por: `-(y * log(p) + (1-y) * log (1-p))` onde y é o verdadeiro rótulo de x e p é a probabilidade estimada por nosso modelo, no caso em que o rótulo é 1.


In [ ]:
def binary_loss(y, p):
    return np.mean(-(y * np.log(p) + (1-y)*np.log(1-p)))

In [ ]:
acts = np.array([1, 0, 0, 1])
preds = np.array([0.9, 0.1, 0.2, 0.8])
binary_loss(acts, preds)

Note que no nosso exemplo acima, nossa precisão é de 100% e nossa perda é de 0,16. Compare isso com uma perda de 0,03 que estamos conseguindo enquanto prevemos cães e gatos. Exercício: jogue com as predições para obter uma perda menor para este exemplo.

Exemplo: Aqui está um exemplo de como calcular a perda(loss) para um exemplo de problema de classificação binária. Suponha que para uma imagem x com rótulo 1 e seu modelo forneça uma previsão de 0,9. Para este caso, a perda(loss) deve ser pequena porque nosso modelo está prevendo um rótulo 1 com alta probabilidade.

`loss = -log(0.9) = 0.10`

Agora, suponha que x tenha rótulo 0, mas nosso modelo está prevendo 0,9. Neste caso, nossa perda deve ser muito maior.

`loss = -log(1-0,9) = 2.30`

Exercício: observe os outros casos e se convença de que isso faz sentido.
Exercício: como você iria reescrever binary_loss usando 'if' em vez de '*' e '+'?
Por que não apenas maximizar a precisão? A perda da classificação binária é uma função mais fácil de otimizar.


## FIM